In [9]:
import sys
import os
import itertools
from torch.utils.data import Dataset, DataLoader, IterableDataset
import torch

from binconvfm.utils.download.gift_eval import list_arrow_files, PostProcessingDataset
from binconvfm.models.lstm import LSTMForecaster

In [18]:
class FirstNDataset(IterableDataset):
    def __init__(self, dataset, n):
        super().__init__()
        self.dataset = dataset
        self.n = n

    def __iter__(self):
        return iter(itertools.islice(self.dataset, self.n))

In [44]:
num_files = 1

pretrain_file_names = list_arrow_files("Salesforce/GiftEvalPretrain")
ds = PostProcessingDataset(file_names=pretrain_file_names[:num_files])
ds = FirstNDataset(ds, 1000)

dataloader = DataLoader(ds, shuffle=False, batch_size=None, num_workers=0)

In [45]:
model = LSTMForecaster(horizon=1, hidden_dim=64, num_epochs=2, logging=True)
model.fit(dataloader)

💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name  | Type | Params | Mode 
---------------------------------------
0 | model | LSTM | 17.2 K | train
---------------------------------------
17.2 K    Trainable params
0         Non-trainable params
17.2 K    Total params
0.069     Total estimated model params size (MB)
3         Modules in train mode
0         Modules in eval mode


Training: |                                                                                     | 0/? [00:00<?…

`Trainer.fit` stopped: `max_epochs=2` reached.


In [46]:
metrics = model.evaluate(dataloader)
metrics

Testing: |                                                                                      | 0/? [00:00<?…

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          crps               86.83729553222656
          mase                      nan
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'mase': nan, 'crps': 86.83729553222656}]

In [47]:
pred = model.predict(dataloader)
len(pred), pred[0].shape

Predicting: |                                                                                   | 0/? [00:00<?…

(1000, torch.Size([64, 1000, 1, 1]))